In [ ]:
#Demo: https://huggingface.co/DAMO-NLP-SG/VideoLLaMA3-7B

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoModel, AutoImageProcessor

In [10]:
import accelerate
print(accelerate.__version__)

1.9.0


In [12]:
!pip install transformers==4.46.3 accelerate==1.0.1
!pip install decord ffmpeg-python imageio opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 16.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/3 [tokenizers]  WARNING: Failed to remove contents in a temporary directory '/n/fs/visualai-scr/temp_LLP/ellie/miniconda3/envs/ellie_env2/lib/python3.10/site-packages/~okenizers'.
  You can safely remove it manually.
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.22m0/3 [tokenizers]
    Uninstalling transformers-4.53.2:90m━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [transformers]
      Successfully uninstalled transformers-4.53.2━━━━━━━━━━━━━━━━ 1/3 [transformers]
  Attempting uninstall: acceleratem━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [transformers]
    Found existing installation: acceler

In [6]:
!pip install accelerate

In [20]:
!which accelerate
!pip list

/n/fs/visualai-scr/temp_LLP/ellie/miniconda3/envs/ellie_env2/bin/accelerate
Package                  Version
------------------------ ------------------
accelerate               1.0.1
annotated-types          0.7.0
antlr4-python3-runtime   4.9.3
asttokens                3.0.0
av                       15.0.0
certifi                  2025.6.15
charset-normalizer       3.4.2
click                    8.2.1
comm                     0.2.2
contourpy                1.3.2
cycler                   0.12.1
debugpy                  1.8.14
decorator                5.2.1
decord                   0.6.0
einops                   0.8.1
exceptiongroup           1.3.0
executing                2.2.0
ffmpeg-python            0.2.0
filelock                 3.18.0
flash_attn               2.8.2
fonttools                4.58.5
fsspec                   2025.5.1
future                   1.0.0
fvcore                   0.1.5.post20221221
gitdb                    4.0.12
GitPython                3.1.44
hf-xet        

In [ ]:
model_name = "DAMO-NLP-SG/VideoLLaMA3-7B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
video_path = "put your video path here"
question = "Describe this video in detail."

# Video conversation
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {"type": "video", "video": {"video_path": video_path, "fps": 1, "max_frames": 128}},
            {"type": "text", "text": question},
        ]
    },
]

inputs = processor(conversation=conversation, return_tensors="pt")
inputs = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
if "pixel_values" in inputs:
    inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)
output_ids = model.generate(**inputs, max_new_tokens=128)
response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(response)

ValueError: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`